In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
# /content/drive/MyDrive/datasets/membrane

In [3]:
# dim 512*512*1

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [5]:
def unet_model(input_shape=(512, 512, 1)):
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Decoder
    up6 = Conv2D(512, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(lr=1e-4), loss=binary_crossentropy, metrics=['accuracy'])

    return model


In [6]:
X_train_loc = '/content/drive/MyDrive/datasets/membrane/train/image/'
Y_train_loc = '/content/drive/MyDrive/datasets/membrane/train/label/'
X_test_loc = '/content/drive/MyDrive/datasets/membrane/test/images'
Y_test_loc = '/content/drive/MyDrive/datasets/membrane/test/labels'

In [7]:
X_train = []
Y_train = []

In [8]:
from skimage.io import imread
from skimage.transform import resize

In [9]:
def load_data(image_path, mask_path, image_size=(512, 512)):
    images = []
    masks = []
    image_files = sorted(os.listdir(image_path))
    mask_files = sorted(os.listdir(mask_path))

    for img_file, mask_file in zip(image_files, mask_files):
        img = load_img(os.path.join(image_path, img_file), color_mode="grayscale", target_size=image_size)
        img = img_to_array(img) / 255.0
        mask = load_img(os.path.join(mask_path, mask_file), color_mode="grayscale", target_size=image_size)
        mask = img_to_array(mask) / 255.0

        images.append(img)
        masks.append(mask)

    images = np.array(images)
    masks = np.array(masks)
    return images, masks

In [10]:
train_images, train_masks = load_data(X_train_loc, Y_train_loc)
test_images, test_masks = load_data(X_test_loc, Y_test_loc)

In [11]:
model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
history = model.fit(train_images, train_masks, epochs=2, batch_size=8, validation_split=0.1)

Epoch 1/2
1/1 [==============================] - 177s 177s/step - loss: 0.6171 - accuracy: 0.7596 - val_loss: 0.6108 - val_accuracy: 0.7301
Epoch 2/2
1/1 [==============================] - 168s 168s/step - loss: 0.5944 - accuracy: 0.7596 - val_loss: 0.5656 - val_accuracy: 0.7301


In [14]:
loss, accuracy = model.evaluate(test_images, test_masks)

1/1 [==============================] - 276s 276s/step - loss: -212.6140 - accuracy: 7.4259e-05
